In [18]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pydantic.dataclasses import dataclass, Field
import dotenv
import interlab
from interlab.queries import query_for_json, query_model
import langchain_community.chat_models
from treetrace import TracingNode
from interlab_zoo.persona.personas import Factory
from interlab_zoo.persona.contacts import link
from interlab_zoo.persona.channels import ChatChannel
from interlab_zoo.resolve.resolve import Situation
from serde.yaml import from_yaml

import langchain
import xml.etree.ElementTree as ET

dotenv.load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [4]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
set_llm_cache(InMemoryCache())


In [5]:
factory = Factory.from_yaml("audience.yaml")

In [6]:

def resolve_direct(resolver, channel=ChatChannel, max_steps_per_player=7):
    players = resolver.players
    link(players)
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message_to_all(sender, others, [resolver.judge])
        if resolver.agreement_check():
            return


def resolve_with_mediator(sitation, mediator_key, channel=ChatChannel, max_steps_per_player=7):
    names = and_join(p.name for p in resolver.players)
    mediator = resolver.factory.create_persona(
        mediator_key,
        public=f"$name is a mediator to arrange arrange an agreement between {names}",
        goal=f"to help {names} find an agreement on {resolver.subject}; you will not watch the movie")
    players = resolver.players
    link(players + [mediator])
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message(sender, mediator, others + [resolver.judge])
        channel.query_and_send_message_to_all(mediator, players, [resolver.judge])
        if resolver.agreement_check():
            return


with TracingNode("root") as root:
    resolver = Resolver("a specific movie name they want to watch together", factory, ["Maria-gpt35", "Connor-gpt35"])

    with TracingNode("resolving"):
        resolve_direct(resolver, "Edward-gpt35")
    print(resolver.agreement_result)
    with TracingNode("scoring"):
        scoring = resolver.score_result()
    df = pd.DataFrame(scoring, columns=["key", "result", "score_basic", "score_wh"])
    print(df)
    
root.display()

NameError: name 'Resolver' is not defined

In [18]:
root.display()

In [ ]:
@dataclass
class Experiment:
    name: str
    people: str
    subject: str
    groups: list[list[str]]

def experiments_from_yaml(path):
    with open(path) as f:
        data = f.read()        
    experiments = from_yaml(list[Experiment], data)
    return {e.name: e for e in experiments}

experiments = experiments_from_yaml("experiments.yaml")
experiments

In [ ]:
def run_experiment(exp):
    with TracingNode(f"Experiment {exp.name}") as root:
        factory = Factory.from_yaml(exp.people)
        situation = Situation(exp.topic, factory, ["Maria-gpt35", "Connor-gpt35"])
    
        with TracingNode("resolving"):
            resolve_direct(resolver, "Edward-gpt35")
        print(resolver.agreement_result)
        with TracingNode("scoring"):
            scoring = resolver.score_result()
        df = pd.DataFrame(scoring, columns=["key", "result", "score_basic", "score_wh"])
        print(df)
    